In [ ]:
import cv2
import torch
import time
import numpy as np

In [ ]:
import torch
import cv2
import numpy as np
import time

# Load YOLOv5 model (replaced with YOLOv5x)
model = torch.hub.load('ultralytics/yolov5', 'yolov5x')  # Load pre-trained YOLOv5x model

# Set video input and output file paths
input_video_path = '/content/Shopping, People, Commerce, Mall, Many, Crowd, Walking   Free Stock video footage   YouTube.mp4'  # Replace with your video path
output_video_path_detection = 'output_detection.avi'  # Output for object detection with crowd count
output_video_path_heatmap = 'output_heatmap.avi'  # Output for heatmap
output_video_path_distancing = 'output_distancing.avi'  # Output for physical distancing detection

# Open input video
cap = cv2.VideoCapture(input_video_path)

# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define video writers for saving the output videos
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Set codec for AVI
out_detection = cv2.VideoWriter(output_video_path_detection, fourcc, fps, (width, height))  # For detection video
out_heatmap = cv2.VideoWriter(output_video_path_heatmap, fourcc, fps, (width, height))  # For heatmap video
out_distancing = cv2.VideoWriter(output_video_path_distancing, fourcc, fps, (width, height))  # For distancing video

# Initialize the heatmap accumulator with zeros
heatmap_accumulator = np.zeros((height, width), dtype=np.float32)

# Define the minimum distance (in pixels) for physical distancing
distance_threshold = 443  # Adjust this value as needed

# Process each frame of the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Start time to calculate reaction time (frame processing time)
    start_time = time.time()

    # Run YOLO object detection on the frame
    results = model(frame)

    # Convert results to a pandas DataFrame
    df = results.pandas().xyxy[0]

    # Filter detections to count only 'person' class
    people = df[df['name'] == 'person']
    people_count = len(people)

    # Calculate reaction time (time taken to process the frame)
    reaction_time = time.time() - start_time

    # Annotate the frame with the people count
    label_count = f'People Count: {people_count}'
    cv2.putText(frame, label_count, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Annotate the frame with the reaction time
    label_time = f'Reaction Time: {reaction_time:.2f}s'
    cv2.putText(frame, label_time, (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    violator_detected = False  # Flag to check if a violator is detected

    # Annotate the frame with the accuracy of detections (confidence scores)
    for index, person in people.iterrows():
        # Extract bounding box and confidence score
        xmin, ymin, xmax, ymax, confidence = int(person['xmin']), int(person['ymin']), int(person['xmax']), int(person['ymax']), float(person['confidence'])

        # Annotate confidence score on the bounding box
        label_accuracy = f'Accuracy: {confidence:.2f}'
        cv2.putText(frame, label_accuracy, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2, cv2.LINE_AA)

        # Draw bounding box around the person
        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

        # Update heatmap with the person’s position (increase the heat in the area where the person is detected)
        heatmap_accumulator[ymin:ymax, xmin:xmax] += 1

    # Check for physical distancing violations
    if people_count > 1:
        for i in range(len(people)):
            for j in range(i + 1, len(people)):
                # Calculate distance between two detected people
                center_i = ((people.iloc[i]['xmin'] + people.iloc[i]['xmax']) // 2, (people.iloc[i]['ymin'] + people.iloc[i]['ymax']) // 2)
                center_j = ((people.iloc[j]['xmin'] + people.iloc[j]['xmax']) // 2, (people.iloc[j]['ymin'] + people.iloc[j]['ymax']) // 2)

                distance = np.sqrt((center_i[0] - center_j[0]) ** 2 + (center_i[1] - center_j[1]) ** 2)

                if distance < distance_threshold:
                    # Draw a red box around the violator
                    cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)  # Red box for violator
                    violator_detected = True  # Mark that a violator has been detected
                    break
            if violator_detected:
                break

    # Normalize the heatmap accumulator to values between 0 and 255
    heatmap_normalized = cv2.normalize(heatmap_accumulator, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

    # Apply a colormap to the heatmap for better visualization
    heatmap_colored = cv2.applyColorMap(heatmap_normalized, cv2.COLORMAP_JET)

    # Overlay the heatmap on the original frame with some transparency
    overlay_frame = cv2.addWeighted(frame, 0.6, heatmap_colored, 0.4, 0)

    # Write the frame to the output videos
    out_detection.write(frame)  # For detection video (with bounding boxes and crowd count)
    out_heatmap.write(overlay_frame)  # For heatmap video (with heatmap overlay)
    out_distancing.write(frame)  # Write distancing frame regardless of violation for consistency

# Release resources
cap.release()
out_detection.release()
out_heatmap.release()
out_distancing.release()  # Release the distancing output video
cv2.destroyAllWindows()

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-10-27 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients, 205.5 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/co